In [1]:
import requests
import pandas as pd
import geopandas as gpd
import json

### Part 1
Scraping School map data from MNPS site (https://mnps.maps.arcgis.com/home/item.html?id=a25c1ef058a247cfa2636a396f3aaedd#data). Table has information for geographic location (latitude and longitude) for each MNPS school. This information will be used for map generation. 

In [2]:
URL = 'https://services.arcgis.com/GBzOxI4K7WHYqcYy/arcgis/rest/services/Schools_Map/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=1000&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D'

In [3]:
response = requests.get(URL)

In [4]:
school_map_data = pd.DataFrame([x['attributes'] for x in response.json()['features']])

In [ ]:
#school_map_data = pd.read_csv('Data/School_map_data.csv')
#school_map_data

In [5]:
school_map_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OBJECTID        171 non-null    int64  
 1   CODE            171 non-null    int64  
 2   SHORT_NAME      171 non-null    object 
 3   SCHOOL_NAME     171 non-null    object 
 4   CLUSTER         171 non-null    int64  
 5   CLUS_NAME       171 non-null    object 
 6   GR_TIER         171 non-null    object 
 7   GRADES          171 non-null    object 
 8   TYPE_1          171 non-null    object 
 9   ZONE_1          171 non-null    object 
 10  NOTES           171 non-null    object 
 11  WEB_LINK        170 non-null    object 
 12  SHORT_ADDR      171 non-null    object 
 13  CITY            171 non-null    object 
 14  ZIP             171 non-null    object 
 15  PHONE           170 non-null    object 
 16  METRO_DIST      171 non-null    int64  
 17  SCHBRD_DIST     171 non-null    int

In [6]:
school_map_data = school_map_data.drop(columns = {'OBJECTID', 'SHORT_NAME', 'GR_TIER', 'NOTES','SCHBRD_DIST','SCHBRD_MBR', 'OPTIONSCH_CURR', 'OPTIONSCH_NEXT', 'OPTID_CURR', 'OPTID_NEXT', 'TN_HOUSE', 'TN_SENATE'})
school_map_data = school_map_data.rename(columns = {'CLUS_NAME':'CLUSTER_NAME','LONG_':'LONGITUDE', 'LAT_':'LATITUDE'})
school_map_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CODE          171 non-null    int64  
 1   SCHOOL_NAME   171 non-null    object 
 2   CLUSTER       171 non-null    int64  
 3   CLUSTER_NAME  171 non-null    object 
 4   GRADES        171 non-null    object 
 5   TYPE_1        171 non-null    object 
 6   ZONE_1        171 non-null    object 
 7   WEB_LINK      170 non-null    object 
 8   SHORT_ADDR    171 non-null    object 
 9   CITY          171 non-null    object 
 10  ZIP           171 non-null    object 
 11  PHONE         170 non-null    object 
 12  METRO_DIST    171 non-null    int64  
 13  LONGITUDE     169 non-null    float64
 14  LATITUDE      169 non-null    float64
 15  CLUSTER_GEO   170 non-null    object 
dtypes: float64(2), int64(3), object(11)
memory usage: 21.5+ KB


In [7]:
school_map_data["SCHOOL_NAME"].unique()

array(['Jere Baxter Middle School', 'Chadwell Elementary School',
       'Stratton Elementary School', 'Madison Middle School',
       'Maplewood High School', 'Liberty Collegiate Academy',
       'Ida B. Wells Elementary School', 'Shwab Elementary School',
       'Tom Joy Elementary School', 'Hattie Cotton Elementary School',
       'Meigs Middle School', 'East Nashville Magnet Middle School',
       'Ross Early Learning Center',
       'KIPP Nashville Collegiate High School',
       'East Nashville Magnet High School',
       'Martin Luther King Jr. School', 'Hume-Fogg High School',
       'Napier Elementary School', 'Cora Howe School',
       'Rosebank Elementary School', 'Lockeland Elementary School',
       'Warner Elementary School', 'Explore Community School',
       'KIPP Academy Nashville Elementary School', 'Nashville Classical',
       'Gateway Elementary School', 'Goodlettsville Middle School',
       'Hull-Jackson Elementary School', 'Haynes Middle School',
       'Isaiah 

In [8]:
school_map_data['ADDRESS'] = school_map_data.SHORT_ADDR.str.cat(school_map_data.CITY, sep=', ')
school_map_data['ADDRESS'] = school_map_data['ADDRESS'].astype(str) + ', TN-'
school_map_data['ADDRESS'] = school_map_data.ADDRESS.str.cat(school_map_data.ZIP, sep='')
school_map_data

,CODE,SCHOOL_NAME,CLUSTER,CLUSTER_NAME,GRADES,TYPE_1,ZONE_1,WEB_LINK,SHORT_ADDR,CITY,ZIP,PHONE,METRO_DIST,LONGITUDE,LATITUDE,CLUSTER_GEO,ADDRESS
0,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,37207,615-262-6710,5,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207"
1,200,Chadwell Elementary School,550,MAPLEWOOD,PK-5,DISTRICT,ZONE SCHOOL,https://chadwell.mnps.org/,321 Port Dr,Madison,37115,615-860-1459,3,-86.741676,36.256377,MAPLEWOOD,"321 Port Dr, Madison, TN-37115"
2,710,Stratton Elementary School,452,HUNTERS LANE,PK-5,DISTRICT,ZONE SCHOOL,https://stratton.mnps.org/,310 Old Hickory Blvd W,Madison,37115,615-860-1486,3,-86.717687,36.265610,HUNTERS LANE,"310 Old Hickory Blvd W, Madison, TN-37115"
3,545,Madison Middle School,452,HUNTERS LANE,6-8,DISTRICT,ZONE SCHOOL,https://madisonmiddle.mnps.org/,300 Old Hickory Blvd,Madison,37115,615-687-4018,3,-86.716452,36.264548,HUNTERS LANE,"300 Old Hickory Blvd, Madison, TN-37115"
4,550,Maplewood High School,550,MAPLEWOOD,9-12,DISTRICT,ZONE SCHOOL,https://maplewood.mnps.org/,401 Walton Ln,Nashville,37216,615-262-6770,5,-86.750472,36.232819,MAPLEWOOD,"401 Walton Ln, Nashville, TN-37216"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,285,John Early Middle School,632,PEARL-COHN,6-8,MAGNET,ZONE SCHOOL,https://johnearly.mnps.org/,1000 Cass St,Nashville,37208,615-291-6369,2,-86.804970,36.187034,PEARL-COHN,"1000 Cass St, Nashville, TN-37208"
167,350,Goodlettsville Elementary School,452,HUNTERS LANE,PK-5,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,575 Joyce Ln,Nashville,37216,615-859-8950,7,NaN,NaN,MAPLEWOOD,"575 Joyce Ln, Nashville, TN-37216"
168,179,Aventura Community School,325,GLENCLIFF,K-1,CHARTER,NOT ZONE SCHOOL,None,3010 Tuggle Ave,Nashville,37211,None,16,NaN,NaN,None,"3010 Tuggle Ave, Nashville, TN-37211"
169,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,37072,615-859-8950,10,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072"


In [9]:
enrollment = pd.read_csv('Data/enrollment.csv')
enrollment

,school,sl,enrollment,years
0,A. Z. Kelley Elementary,Elementary School,621,07-08
1,Alex Green Elementary,Elementary School,286,07-08
2,Amqui Elementary,Elementary School,667,07-08
3,Andrew Jackson Elementary,Elementary School,556,07-08
4,Antioch High School,High School,2403,07-08
...,...,...,...,...
2508,West End Middle,Middle School,343,22-23
2509,Westmeade Elementary,Elementary School,373,22-23
2510,Whites Creek High School,High School,601,22-23
2511,William Henry Oliver Middle,Middle School,669,22-23


In [10]:
enrollment = enrollment.rename(columns = {'school': 'SCHOOL_NAME', 'sl':'SCHOOL_TYPE', 'enrollment' : 'TOTAL_ENROLLMENT', 'years' : 'YEARS'})
enrollment

,SCHOOL_NAME,SCHOOL_TYPE,TOTAL_ENROLLMENT,YEARS
0,A. Z. Kelley Elementary,Elementary School,621,07-08
1,Alex Green Elementary,Elementary School,286,07-08
2,Amqui Elementary,Elementary School,667,07-08
3,Andrew Jackson Elementary,Elementary School,556,07-08
4,Antioch High School,High School,2403,07-08
...,...,...,...,...
2508,West End Middle,Middle School,343,22-23
2509,Westmeade Elementary,Elementary School,373,22-23
2510,Whites Creek High School,High School,601,22-23
2511,William Henry Oliver Middle,Middle School,669,22-23


In [11]:
enrollment["SCHOOL_NAME"].unique()

array(['A. Z. Kelley Elementary', 'Alex Green Elementary',
       'Amqui Elementary', 'Andrew Jackson Elementary',
       'Antioch High School', 'Antioch Middle', 'Apollo Middle',
       'Bailey Middle', 'Bellevue Middle', 'Bellshire Elementary',
       'Bordeaux Enhanced Option School', 'Brick Church Middle School',
       'Brookmeade Elementary School', 'Buena Vista Elementary',
       'Cameron Middle School', 'Carter-Lawrence Elementary',
       'Chadwell Elementary', 'Charlotte Park Elementary',
       'Cockrill Elementary', 'Cohn Adult Learning Center',
       'Cohn Alternative Center', 'Cole Elementary', 'Cora Howe School',
       'Creswell Middle School of the Arts', 'Crieve Hall Elementary',
       'Croft Middle', 'Cumberland Elementary', 'Dalewood Middle School',
       'Dan Mills Elementary', 'Dodson Elementary', 'Donelson Middle',
       'Dupont Elementary', 'Dupont-Hadley Middle', 'Dupont-Tyler Middle',
       'Eakin Elementary', 'Early College High School',
       'East Na

In [ ]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].replace({'Antioch High School': 'Antioch High' ,
                                   'Whites Creek High School': 'Whites Creek High', 
                                   'Bordeaux Enhanced Option School' : 'Bordeaux Enhanced Option',
                                   'Brick Church Middle School' : 'Brick Church Middle',
                                   'Brookmeade Elementary School' : 'Brookmeade Elementary',
                                   'Cameron Middle School' : 'Cameron Middle',
                                    'Cora Howe School' : 'Cora Howe', 
                                    'Dalewood Middle School' : 'Dalewood Middle', 
                                    'Early College High School' : 'Early College High', 
                                    'East Nashville School' : 'East Nashville', 
                                    'Ewing Park Middle School' : 'Ewing Park Middle', 
                                    'Glencliff High School' : 'Glencliff High',
                                    'H.G. Hill Middle School' : 'H.G. Hill Middle', 
                                    'Martha Vaught Middle School': 'Martha Vaught Middle', 
                                   'Martin Luther King Jr School': 'Martin Luther King Jr',
                                   'May Werthan Shayne Elementary School' : 'May Werthan Shayne Elementary',
                                     'Ross Elementary School' : 'Ross Elementary', 
                                    'Stratford STEM Magnet School': 'Stratford STEM Magnet', 
                                    'W.A. Bass Middle School' : 'W.A. Bass Middle', 
                                     'Cane Ridge High School' :  'Cane Ridge High', 
                                    'Murrell School' :  'Murrell', 
                                    'Nashville Diploma Plus High School' : 'Nashville Diploma Plus High', 
                                    'Smithson Craighead Middle School' : 'Smithson Craighead Middle', 
                                    'Transition School' : 'Transition', 
                                    'East End Preparatory School' : 'East End Preparatory',
                                   'Metro Nashville Virtual School' : 'Metro Nashville Virtual',
                                    'Robertson Academy Gifted School' : 'Robertson Academy Gifted',
                                    'The Cohn School' : 'The Cohn',
                                    'KIPP Nashville Collegiate High School': 'KIPP Nashville Collegiate High',
                                    'Explore Community School' : 'Explore Community',
                                    'Knowledge Academies High School' : 'Knowledge Academies High',
                                    'STEM Prep High School' : 'STEM Prep High', 
                                    'RePublic High School' : 'RePublic High', 
                                    'Smith Springs Elementary School' : 'Smith Springs Elementary',                            
                                    'Aventura Community School' : 'Aventura Community',
                                    'Nashville Big Picture High School' : 'Nashville Big Picture High',
                                    'Waverly-Belmont Elementary School' : 'Waverly-Belmont Elementary',
                                    'Johnson School' : 'Johnson'                     })

In [ ]:
enrollment["SCHOOL_NAME"].unique()

In [ ]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].astype(str) + ' School'
enrollment

In [ ]:
enrollment["SCHOOL_NAME"].unique()

In [ ]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].replace({
 'Liberty Collegiate Academy School' :  'Liberty Collegiate Academy',   
'Ross Early Learning Center School': 'Ross Early Learning Center',
'Nashville Classical School': 'Nashville Classical',
'Purpose Prep School': 'Purpose Prep', 
'Rocketship Nashville Northeast Elementary School':'Rocketship Nashville Northeast Elementary',
'Ivanetta H. Davis Early Learning Center at Bordeaux School': 'Ivanetta H. Davis Early Learning Center at Bordeaux',
'Stratford STEM Magnet School - Upper/Lower Campus School': 'Stratford STEM Magnet School - Upper/Lower Campus',
'Smithson Craighead Academy School':'Smithson Craighead Academy',
'LEAD Prep Southeast School': 'LEAD Prep Southeast',
'Strive Collegiate Academy School':'Strive Collegiate Academy',
'The Academy at Opry Mills School':'The Academy at Opry Mills',
'Rocketship United Academy School':'Rocketship United Academy',
'Harris-Hillman Special Education School':'Harris-Hillman Special Education',
'Johnson Alternative Learning Center School':'Johnson Alternative Learning Center',
'Nashville School of the Arts School':'Nashville School of the Arts',
'Cameron College Preparatory School':'Cameron College Preparatory',
'Casa Azafran Early Learning Center School' :'Casa Azafran Early Learning Center', 
'STEM Prep Academy School' :'STEM Prep Academy',
'LEAD Academy High School':'LEAD Academy High',
'The Academy at Old Cockrill School' :'The Academy at Old Cockrill',
'W. A. Bass Learning Center - Adult School' : 'W. A. Bass Learning Center - Adult',
'W. A. Bass Learning Center - Transitions School':'W. A. Bass Learning Center - Transitions',
'Nashville Prep School' : 'Nashville Prep', 
'Tusculum Elementary School' :'Tusculum Elementary', 
'Valor Flagship Academy School' :'Valor Flagship Academy',
'Valor Voyager Academy School' :'Valor Voyager Academy',
'John Trotwood Moore Middle School': 'John Trotwood Moore Middle',
'Robertson Academy School':  'Robertson Academy', 
'KIPP Academy Nashville School':'KIPP Academy Nashville',
'Martin Professional Center School' : 'Martin Professional Center',
'Board of Education School' :'Board of Education',
'The Academy at Hickory Hollow School' : 'The Academy at Hickory Hollow',
'Cambridge Early Learning Center School': 'Cambridge Early Learning Center',
'KA @ The Crossings School' :'KA @ The Crossings', 
'KIPP Nashville College Prep School' : 'KIPP Nashville College Prep',
'KIPP Nashville College Prep Elementary School':'KIPP Nashville College Prep Elementary',
'W. A. Bass Learning Center - ALC School' : 'W. A. Bass Learning Center - ALC',
'Carlisle Beasley, Jr Building (Transportation) School' : 'Carlisle Beasley, Jr Building (Transportation)',
'MNPS Supply Center School' : 'MNPS Supply Center',
'MNPS Spencer Lane Warehouse School' : 'MNPS Spencer Lane Warehouse',
'McGruder Family Resource Center School':'McGruder Family Resource Center', 
'MNPS Maint/Supply - GENESCO School' : 'MNPS Maint/Supply - GENESCO',
'Nashville Collegiate Prep School' : 'Nashville Collegiate Prep' 
})

In [ ]:
school_enrollment = pd.merge(left = school_map_data, 
         right = enrollment, 
         on = 'SCHOOL_NAME',
        how = 'inner'            
        )

In [ ]:
school_enrollment.head(20)

In [ ]:
school_enrollment["SCHOOL_NAME"]. nunique()

Exporting data frame into csv file for use in R 

In [ ]:
School_map_data.to_csv('School_map_data.csv', index = True)

### Part 2
Using the American Community Survey API, obtained for each census tract, the population (B01001_001E in the detailed tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37.


In [ ]:
endpoint = 'https://api.census.gov/data/2021/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2021 = requests.get(endpoint)
Population2021 = Pop_response2021.json()
Population2021_df =pd.DataFrame.from_records(Population2021)
Population2021_df

In [ ]:
Population2021_df = Population2021_df.rename(columns = Population2021_df.iloc[0])
Population2021_df = Population2021_df.drop(0)
Population2021_df = Population2021_df.rename(columns = { "B01001_001E" : "2021"})
Population2021_df = Population2021_df.drop(columns = ['state', 'county'])
Population2021_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2020 = requests.get(endpoint)
Population2020 = Pop_response2020.json()
Population2020_df =pd.DataFrame.from_records(Population2020)
Population2020_df

In [ ]:
Population2020_df = Population2020_df.rename(columns = Population2020_df.iloc[0])
Population2020_df = Population2020_df.drop(0)
Population2020_df = Population2020_df.rename(columns = { "B01001_001E" : "2020"})
Population2020_df= Population2020_df.drop(columns = ['state', 'county'])
Population2020_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2019 = requests.get(endpoint)
Population2019 = Pop_response2019.json()
Population2019_df =pd.DataFrame.from_records(Population2019)
Population2019_df

In [ ]:
Population2019_df = Population2019_df.rename(columns = Population2019_df.iloc[0])
Population2019_df = Population2019_df.drop(0)
Population2019_df = Population2019_df.rename(columns = { "B01001_001E" : "2019"})
Population2019_df = Population2019_df.drop(columns = ['state', 'county'])
Population2019_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2018 = requests.get(endpoint)
Population2018 = Pop_response2018.json()
Population2018_df = pd.DataFrame.from_records(Population2018)
Population2018_df

In [ ]:
Population2018_df = Population2018_df.rename(columns = Population2018_df.iloc[0])
Population2018_df = Population2018_df.drop(0)
Population2018_df = Population2018_df.rename(columns = { "B01001_001E" : "2018"})
Population2018_df = Population2018_df.drop(columns = ['state', 'county'])
Population2018_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2017 = requests.get(endpoint)
Population2017 = Pop_response2017.json()
Population2017_df = pd.DataFrame.from_records(Population2017)
Population2017_df

In [ ]:
Population2017_df = Population2017_df.rename(columns = Population2017_df.iloc[0])
Population2017_df = Population2017_df.drop(0)
Population2017_df = Population2017_df.rename(columns = { "B01001_001E" : "2017"})
Population2017_df = Population2017_df.drop(columns = ['state', 'county'])
Population2017_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2016 = requests.get(endpoint)
Population2016 = Pop_response2016.json()
Population2016_df = pd.DataFrame.from_records(Population2016)
Population2016_df

In [ ]:
Population2016_df = Population2016_df.rename(columns = Population2016_df.iloc[0])
Population2016_df = Population2016_df.drop(0)
Population2016_df = Population2016_df.rename(columns = { "B01001_001E" : "2016"})
Population2016_df = Population2016_df.drop(columns = ['state', 'county'])
Population2016_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2015/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2015 = requests.get(endpoint)
Population2015 = Pop_response2015.json()
Population2015_df = pd.DataFrame.from_records(Population2015)
Population2015_df

In [ ]:
Population2015_df = Population2015_df.rename(columns = Population2015_df.iloc[0])
Population2015_df = Population2015_df.drop(0)
Population2015_df = Population2015_df.rename(columns = { "B01001_001E" : "2015"})
Population2015_df = Population2015_df.drop(columns = ['state', 'county'])
Population2015_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2014/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2014 = requests.get(endpoint)
Population2014 = Pop_response2014.json()
Population2014_df = pd.DataFrame.from_records(Population2014)
Population2014_df

In [ ]:
Population2014_df = Population2014_df.rename(columns = Population2014_df.iloc[0])
Population2014_df = Population2014_df.drop(0)
Population2014_df = Population2014_df.rename(columns = { "B01001_001E" : "2014"})
Population2014_df = Population2014_df.drop(columns = ['state', 'county'])
Population2014_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2013/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2013 = requests.get(endpoint)
Population2013 = Pop_response2013.json()
Population2013_df = pd.DataFrame.from_records(Population2013)
Population2013_df

In [ ]:
Population2013_df = Population2013_df.rename(columns = Population2013_df.iloc[0])
Population2013_df = Population2013_df.drop(0)
Population2013_df = Population2013_df.rename(columns = { "B01001_001E" : "2013"})
Population2013_df = Population2013_df.drop(columns = ['state', 'county'])
Population2013_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2012/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2012 = requests.get(endpoint)
Population2012 = Pop_response2012.json()
Population2012_df = pd.DataFrame.from_records(Population2012)
Population2012_df

In [ ]:
Population2012_df = Population2012_df.rename(columns = Population2012_df.iloc[0])
Population2012_df = Population2012_df.drop(0)
Population2012_df = Population2012_df.rename(columns = { "B01001_001E" : "2012"})
Population2012_df = Population2012_df.drop(columns = ['state', 'county'])
Population2012_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2011/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2011 = requests.get(endpoint)
Population2011 = Pop_response2011.json()
Population2011_df = pd.DataFrame.from_records(Population2011)
Population2011_df

In [ ]:
Population2011_df = Population2011_df.rename(columns = Population2011_df.iloc[0])
Population2011_df = Population2011_df.drop(0)
Population2011_df = Population2011_df.rename(columns = { "B01001_001E" : "2011"})
Population2011_df = Population2011_df.drop(columns = ['state', 'county'])
Population2011_df 

In [ ]:
endpoint= 'https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2010 = requests.get(endpoint)
Population2010 = Pop_response2010.json()
Population2010_df = pd.DataFrame.from_records(Population2010)
Population2010_df

In [ ]:
Population2010_df = Population2010_df.rename(columns = Population2010_df.iloc[0])
Population2010_df = Population2010_df.drop(0)
Population2010_df = Population2010_df.rename(columns = { "B01001_001E" : "2010"})
Population2010_df = Population2010_df.drop(columns = ['state', 'county'])
Population2010_df 

In [ ]:
endpoint= 'https://api.census.gov/data/2009/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2009 = requests.get(endpoint)
Population2009= Pop_response2009.json()
Population2009_df=pd.DataFrame.from_records(Population2009)
Population2009_df

In [ ]:
Population2009_df = Population2009_df.rename(columns = Population2009_df.iloc[0])
Population2009_df = Population2009_df.drop(0)
Population2009_df = Population2009_df.rename(columns = { "B01001_001E" : "2009"})
Population2009_df = Population2009_df.drop(columns = ['state', 'county'])
Population2009_df 

In [ ]:
Population = pd.merge(left = Population2021_df, 
         right = Population2020_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2019_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2018_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2017_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2016_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2015_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2014_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2013_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2012_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2011_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2010_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population.to_csv('population_census_tract.csv', index = True)

In [ ]:
import re